In [1]:
from ibl_pipeline.analyses import behavior as behavior_analyses
from ibl_pipeline import behavior
from ibl_pipeline import subject, acquisition
from ibl_pipeline.utils import psychofit as psy
from uuid import UUID
import numpy as np
import datetime
import datajoint as dj
import json
import seaborn as sns
import pandas as pd
import plotly
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

Connecting shan@datajoint.internationalbrainlab.org:3306


In [51]:
subj = subject.Subject & {'subject_nickname': 'CSHL_007'}
# get trial counts and session length to date
session_info = (behavior.TrialSet * acquisition.Session & subj).proj(
    'n_trials', session_date='DATE(session_start_time)',
    session_duration='TIMESTAMPDIFF(MINUTE, session_start_time, session_end_time)').fetch(as_dict=True)
session_info = pd.DataFrame(session_info)
session_info = session_info.where((pd.notnull(session_info)), None)

n_trials = session_info['n_trials'].tolist()
max_trials = max(n_trials)
yrange = [0, max_trials+50]

# get the first date when animal became "trained" and "ready for ephys"
first_trained = subj.aggr(
    behavior_analyses.SessionTrainingStatus & 'training_status = "trained"', 
    first_session='DATE(min(session_start_time))')
first_biased = subj.aggr(
    behavior_analyses.SessionTrainingStatus & 'training_status = "ready for ephys"', 
    first_session='DATE(min(session_start_time))')

In [52]:
if len(first_trained):
    first_trained_date = first_trained.fetch1('first_session').strftime('%Y-%m-%d')

if len(first_biased):
    first_biased_date = first_biased.fetch1('first_session').strftime('%Y-%m-%d')

In [58]:
trial_counts = go.Scatter(
    x=[t.strftime('%Y-%m-%d') for t in session_info['session_date'].tolist()],
    y=session_info['n_trials'].tolist(),
    mode='markers+lines',
    marker=dict(
        size=6,
        color='black'),
    name='trial counts',
    yaxis='y1'
)

if len(first_trained):
    first_trained_plot=go.Scatter(
        x=[first_trained_date, first_trained_date],
        y=yrange,
        mode="lines",
        marker=dict(
            color='orange'),
        name='first day got trained'
    )

if len(first_biased):
    first_biased_plot=go.Scatter(
        x=[first_biased_date, first_biased_date],
        y=yrange,
        mode="lines",
        marker=dict(
            color='green'),
        name='first day got biased'
    )
session_length = go.Scatter(
    x=[t.strftime('%Y-%m-%d') for t in session_info['session_date'].tolist()],
    y=session_info['session_duration'].tolist(),
    mode='markers+lines',
    marker=dict(
        size=6,
        color='red'),
    name='session duration',
    yaxis='y2'
)

data = [trial_counts, session_length]

if len(first_trained):
    data.append(first_trained_plot)

if len(first_biased):
    data.append(first_biased_plot)

layout = go.Layout(
    yaxis=dict(
        title='Trial counts',
        range=yrange
    ),
    yaxis2=dict(
        title='Session duration (mins)',
        overlaying='y',
        color='red',
        side='right'
    ),
    xaxis={'title':'Date'}, 
    width=500,
    height=400,
    title='Trial counts and session duration',
    legend=dict(
        x=0, 
        y=1.2,
        orientation='h'),
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [59]:
f = open("trial_counts_session_duration.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()